### Imports and utility

In [ ]:
# Standard library imports
import os
import sys
import importlib
import numpy as np
import random
import cv2
from tqdm import tqdm

# Add project-specific paths
sys.path.append(os.path.join(os.getcwd(), 'common'))

# Import custom modules
import utility
import embedding
import detection
import grt

# Reload module for updates
importlib.reload(grt)

In [2]:
def points(avg):
    if avg >= 53:
        points = 0
    elif avg >= 50:
        points = 1
    elif avg >= 47:
        points = 2
    elif avg >= 44:
        points = 3
    elif avg >= 41:
        points = 4
    elif avg >= 38:
        points = 5
    else:
        points = 6
    return points

### Embedding

In [5]:
# Randomly select 10 images
img_folder = 'sample_imgs'
img_files =  [f for f in os.listdir(img_folder) if f.endswith(('.bmp'))]
img_files = random.sample(img_files, 1)
images = []
for file in img_files:
    img_path = os.path.join(img_folder, file)
    images.append(cv2.imread(img_path, 0))
images = np.array(images) # optional

mark = np.load('ammhackati.npy')

alpha = 0.5

watermarked = []
for img in images:
    watermarked.append(embedding.embedding(img, mark, alpha))

### Global attacks

1. Blur using gaussian filtering

In [ ]:
progress_bar = tqdm(watermarked, desc = "Blurring 10 images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for sigma in np.arange(0, 5, 0.1, dtype = np.float16):
        attacked, attack_name, usd = grt.blur_gauss(wm, sigma)
        detected = detection.detection(images[index], wm, attacked, alpha, 2)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("Average wpsnr of successful attacks: " + str(avg) + ' --> ' + str(points(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: ", max) 
else:
    print("This attack is not very effective...")
# for h in history:
#     print(h)   
for s in successful_attacks:
    print(s) 

2. Blur using median filter

In [ ]:
progress_bar = tqdm(watermarked, desc = "Blurring 10 images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for height in np.arange(1, 15, 2):
        for width in np.arange(1, 15, 2):
            attacked, attack_name, usd = grt.blur_median(wm, [height, width])
            detected = detection.detection(images[index], wm, attacked, alpha, 2)
            wpsnr_attacked = utility.wpsnr(wm, attacked)
            log = {"image" : img_files[index],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}
            history.append(log)
            if detected == 0:
                total += wpsnr_attacked
                succ +=1
                successful_attacks.append(log)
                if wpsnr_attacked > max:
                    max = wpsnr_attacked
                break

if succ > 0:
    avg = total/succ
    print("Average wpsnr of successful attacks: " + str(avg) + ' --> ' + str(points(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: ", max) 
else:
    print("This attack is not very effective...")
# for h in history:
#     print(h)   
for s in successful_attacks:
    print(s)

3. Adding white gaussian noise (AWGN)

In [ ]:
progress_bar = tqdm(watermarked, desc = "Applying awgn on 10 images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for std in np.arange(0, 1, 0.2, dtype = np.float16):
        attacked, attack_name, usd = grt.awgn(wm, 0, std, 100)
        detected = detection.detection(images[index], wm, attacked, alpha, 2)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("Average wpsnr of successful attacks: " + str(avg) + ' --> ' + str(points(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: ", max) 
else:
    print("This attack is not very effective...")
# for h in history:
#     print(h)   
for s in successful_attacks:
    print(s)

4. JPEG compression

In [ ]:
progress_bar = tqdm(watermarked, desc = "Compressing 10 images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for qf in range(1, 100, 2):
        attacked, attack_name, usd = grt.jpeg_compression(wm, (100 - qf))
        detected = detection.detection(images[index], wm, attacked, alpha, 2)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("Average wpsnr of successful attacks: " + str(avg) + ' --> ' + str(points(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: ", max) 
else:
    print("This attack is not very effective...")
# for h in history:
#     print(h)   
for s in successful_attacks:
    print(s)

5. Resizing

In [ ]:
progress_bar = tqdm(watermarked, desc = "Resizing 10 images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for scale in np.arange(0, 1, 0.001, dtype = np.float16):
        attacked, attack_name, usd = grt.resize(wm, (1 - scale))
        detected = detection.detection(images[index], wm, attacked, alpha, 2)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("Average wpsnr of successful attacks: " + str(avg) + ' --> ' + str(points(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: ", max) 
else:
    print("This attack is not very effective...")
# for h in history:
#     print(h)   
for s in successful_attacks:
    print(s) 